In [48]:
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as pltkeras

import keras
from keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense
from keras.models import Model, model_from_json
from keras.optimizers import SGD
from keras.applications import inception_v3
from keras.datasets import cifar10
from keras.utils import np_utils
from keras import backend as K

import time

In [5]:
(train_data, train_labels), (test_data, test_labels) = cifar10.load_data()

In [6]:
train_data = train_data.astype('float32')
test_data = test_data.astype('float32')
train_data = train_data / 255.0
test_data = test_data / 255.0

In [9]:
train_labels = np_utils.to_categorical(train_labels)
test_labels = np_utils.to_categorical(test_labels)

In [28]:
input_img = Input(shape = (32, 32, 3))

In [29]:
tower_1 = Conv2D(64, (1,1), padding='same', activation='relu')(input_img)
tower_1 = Conv2D(64, (3,3), padding='same', activation='relu')(tower_1)

tower_2 = Conv2D(64, (1,1), padding='same', activation='relu')(input_img)
tower_2 = Conv2D(64, (5,5), padding='same', activation='relu')(tower_2)

tower_3 = MaxPool2D((3,3), strides=(1,1), padding='same')(input_img)
tower_3 = Conv2D(64, (1,1), padding='same', activation='relu')(tower_3)

In [30]:
output = keras.layers.concatenate([tower_1, tower_2, tower_3], axis = 3)

In [31]:
output = Flatten()(output)

In [32]:
out = Dense(10, activation='softmax')(output)

In [36]:
inception_model = Model(inputs = input_img, outputs = out)
inception_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 32, 32, 64)   256         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 32, 32, 64)   256         input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, 32, 32, 3)    0           input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_7 (

In [42]:
epochs = 1
lrate = 0.01
decay = lrate/ epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)

In [43]:
inception_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [44]:
inception_model.fit(train_data, train_labels, validation_data=(test_data, test_labels), epochs=epochs, batch_size=32)

Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 1280s 26ms/step - loss: 1.3767 - acc: 0.5165 - val_loss: 1.2964 - val_acc: 0.5387


In [45]:
scores = inception_model.evaluate(test_data, test_labels, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

10000/10000 [==============================] - 63s 6ms/step
Accuracy: 53.87%


In [49]:
import os
model_json = inception_model.to_json()

with open("models/inception_model.json", "w") as json_file:
    json_file.write(model_json)
    
inception_model.save_weights(os.path.join(os.getcwd(), 'models/inception_model.h5'))

<hr/>